## Imports

In [1]:
# General Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
import time
from datetime import date
from datetime import datetime
import sys

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.proxy import *
from selenium.webdriver.common.proxy import Proxy, ProxyType

# processing
#!pip install tldextract #-> install package
import tldextract
from urllib.parse import urlparse

# Scrapping "simple"

In [10]:
########## SELENIUM ##########

In [2]:
def recaptcha(driver):
    try:
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[title='reCAPTCHA']")))
        print('*** reCaptcha détécté ***')
        print('*** 1min pour résoudre le puzzle avant suite du script ***\n')
        time.sleep(60)
    except:
        pass

In [3]:
def scrapator(search_terms, nb_pages): # à ajouter -> proxy, langue, headless browser...
    
    """
    -----------------------------------------
    fonction pour scrapper serp google (SEO only)
    -----------------------------------------
    search_terms -> liste de search terms à utiliser
    nb_pages -> nombre de pages à scrapper (int)
    visible -> si on veut visualiser le browser ou non - par défaut = True donc visible (bool)
    (proxy -> proxy à utiliser (WIP))
    
    return -> df
    """
    
    start_time = datetime.now() # à formater
    print(f'Début du programme : {start_time}\n') # à formater
    
    # AJOUTER GESTION ERREUR QUELQUEPART
    
    df = pd.DataFrame()
    
    # initier webdriver
    options = webdriver.ChromeOptions()
    #if visible is False:
    #    options.add_argument("--headless")
    driver = webdriver.Chrome(options = options)
    
    #action = ActionChains(driver) -> à enlever ? 
    
    print('***** scrapping en cours *****')
    
    # boucle sur search terms
    for term in search_terms:
        
        print(f'\n**** search term - {term} ****\n')
        
        # aller sur google
        driver.get("https://google.com")
        try:
            # accepter blablabla
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#L2AGLb.tHlp8d"))).click()
        except:
            pass
        
        # petite pause
        time.sleep(round(random.uniform(1, 10), 1))
    
        # trouver la search bar
        elem = driver.find_element(By.NAME, "q")
        elem.clear() # pour la vider si elle est pas vide
    
        # entrer element de recherche dans la search bar
        elem.send_keys(term)
        time.sleep(2) # à ne pas oublier sinon va juste aller à la ligne après le mot sans faire entrer
        elem.send_keys(Keys.RETURN)
    
        # petite pause
        time.sleep(round(random.uniform(1, 10), 1))
    
        textes = []
        titres = []
        url = []
        image = []
        page_pos = []
        pages = []
        seaseo = []
        n = 0

        # prendre les infos des nb_pages pages
        while n < nb_pages:
            
            # compteur de page
            n += 1
            print(f'*** -- scrapping page {n} -- ***')
            
            # captcha
            recaptcha(driver)

            # trouver tous les articles d'une page
            driver.save_screenshot('screenshot.png')
            articles_SEO = driver.find_elements(By.CLASS_NAME, "MjjYud") # -> SEO
            articles_SEA = driver.find_elements(By.CLASS_NAME, "uEierd") # -> SEA
            position = 0
    
            # boucler sur articles SEO
            for article in articles_SEO:
                
                try:
                    # titre
                    title = article.find_element(By.TAG_NAME, 'h3') # quand pas de titre pour le seo -> pas vrai article seo
                    titres.append(title.text)
                    
                    # position
                    position += 1
                    page_pos.append(position)
        
                    # texte
                    textes.append(article.text)
                    
                    # liens
                    link = article.find_element(By.TAG_NAME, 'a')
                    url.append(link.get_attribute("href"))
                    
                    # page
                    pages.append(n)
                    
                    # type de contenu
                    seaseo.append('SEO')
                    
                    # Image
                    try:
                        img = article.find_element(By.CLASS_NAME, 'Z26q7c.UK95Uc.Sth6v')
                        image.append('Oui')
                    except:
                        image.append('Non')
                    
                except:
                    titres.append('vide')
                    page_pos.append('vide')
                    textes.append('vide')
                    url.append('vide')
                    pages.append('vide')
                    image.append('vide')
                    seaseo.append('SEO vide')
    
            position = 0    
            # boucler sur articles SEA
            for add in articles_SEA:
                
                # position
                position += 1
                page_pos.append(position)
        
                # texte
                textes.append(add.text)
        
                # titre
                try:
                    title = add.find_element(By.CLASS_NAME, 'CCgQ5.vCa9Yd.QfkTvb.N8QANc.MUxGbd.v0nnCb')
                    titres.append(title.text)
                except:
                    titres.append('erreur')

                # liens
                try:
                    link = add.find_element(By.TAG_NAME, 'a')
                    url.append(link.get_attribute("href"))
                except:
                    url.append('erreur')
                
                # page
                pages.append(n)
                
                # type de contenu
                seaseo.append('SEA')
                
                try:
                    img = add.find_element(By.CLASS_NAME, 'g-img.ZGomKf')
                    image.append('Oui')
                except:
                    image.append('Non')

            
            # se déplacer en bas de la page
            bottom_element = driver.find_element(By.ID, 'botstuff')
            bottom_element.location_once_scrolled_into_view
    
            # changer de page
            try:
                driver.find_element(By.XPATH, ("//*[contains(text(),'Suivant')]")).click()
            except:
                pass
        
            # petite pause
            time.sleep(round(random.uniform(1, 10), 1))
        
        # définir le bon indice à prendre quand pas le même nombre d'élements

        if len(titres) <= len(url) and len(titres) <= len(textes):
            size = len(titres)
        elif len(textes) <= len(url) and len(textes) <= len(titres):
            size = len(textes)
        elif len(url) <= len(textes) and len(url) <= len(titres):
            size = len(url)
        elif len(url) == len(textes) and len(url) == len(titres) and len(textes) == len(titres):
            print('même taille')
            size = len(url)
        else:
            print('problème quelque part')
            size = 0
        
        # mettre le tout dans un df
        df_term = pd.DataFrame()
        df_term['url'] = url[0:size]
        df_term['titles'] = titres[0:size]
        df_term['texte'] = textes[0:size]
        df_term['search_term'] = term
        df_term['pages'] = pages[0:size]
        df_term['page_position'] = page_pos[0:size]
        df_term['type'] = seaseo[0:size]
        df_term['images'] = image[0:size]
    
        df_term.drop(df_term.loc[df_term['titles'] == ''].index, inplace = True)
        df_term.reset_index(inplace = True, drop = True)
    
        df_term['company'] = df_term['url'].apply(lambda x : tldextract.extract(x).domain)
        df_term['domain'] = df_term['url'].apply(lambda x : urlparse(x).netloc)
        #df_term['serp_position'] = np.arange(1, df_term.shape[0] + 1)
        
        # Merger avec les données des messages précédents
        df = pd.concat([df, df_term], ignore_index = True)
        df['Date'] = date.today().strftime("%d/%m/%y")
        
        print(f'\nFin du scrapping pour le search term : {term} ')
    
    # quitter driver
    driver.close()
    
    end_time = datetime.now() # à formater
    print('Duration: {}'.format(end_time - start_time)) # à formater
    
    return df

In [4]:
%%time

# test scraping 5 pages
mots = ['monitoring reporting verification soil', 'soil monitoring reporting']

df = scrapator(mots, 5)

Début du programme : 2023-11-01 12:58:19.530897



WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:
df

,url,titles,texte,search_term,pages,page_position,type,images,company,domain,Date
0,https://scholar.google.fr/scholar?q=monitoring...,Articles universitaires correspondant aux term...,Articles universitaires correspondant aux term...,monitoring reporting verification soil,1,1,SEO,Non,google,scholar.google.fr,30/10/23
1,https://www.horizon-europe.gouv.fr/monitoring-...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",monitoring reporting verification soil,1,2,SEO,Non,horizon-europe,www.horizon-europe.gouv.fr,30/10/23
2,https://www.fao.org/documents/card/en?details=...,"A protocol for measurement, monitoring, report...","A protocol for measurement, monitoring, report...",monitoring reporting verification soil,1,4,SEO,Non,fao,www.fao.org,30/10/23
3,https://gi.copernicus.org/articles/11/93/2022/,"Towards agricultural soil carbon monitoring, r...","Towards agricultural soil carbon monitoring, r...",monitoring reporting verification soil,1,5,SEO,Non,copernicus,gi.copernicus.org,30/10/23
4,https://cordis.europa.eu/project/id/101112754,"Monitoring, Reporting and Verification of Soil...","Monitoring, Reporting and Verification of Soil...",monitoring reporting verification soil,1,6,SEO,Non,europa,cordis.europa.eu,30/10/23
...,...,...,...,...,...,...,...,...,...,...,...
200,https://www.databridgemarketresearch.com/repor...,Global Soil Monitoring Market – Industry Trend...,Global Soil Monitoring Market – Industry Trend...,soil monitoring reporting,5,18,SEO,Non,databridgemarketresearch,www.databridgemarketresearch.com,30/10/23
201,https://www.marketsandmarkets.com/Market-Repor...,"Soil Monitoring Market Size, Share, Trends, an...","Soil Monitoring Market Size, Share, Trends, an...",soil monitoring reporting,5,19,SEO,Non,marketsandmarkets,www.marketsandmarkets.com,30/10/23
202,https://forest.moscowfsl.wsu.edu/smp/solo/GeoP...,Idaho Panhandle National Forest: Soil Monitori...,Idaho Panhandle National Forest: Soil Monitori...,soil monitoring reporting,5,20,SEO,Non,wsu,forest.moscowfsl.wsu.edu,30/10/23
203,https://environment.govt.nz/publications/land-...,Land and soil monitoring: A guide for SoE and ...,Land and soil monitoring: A guide for SoE and ...,soil monitoring reporting,5,21,SEO,Non,environment,environment.govt.nz,30/10/23


In [ ]:
df.loc[df['titles'] == 'vide']

,url,titles,texte,search_term,pages,page_position,type,images,company,domain,Date
10,vide,vide,vide,iphone reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
11,vide,vide,vide,iphone reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
27,vide,vide,vide,iphone reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
44,vide,vide,vide,iphone reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
60,vide,vide,vide,iphone reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
...,...,...,...,...,...,...,...,...,...,...,...
717,vide,vide,vide,mac reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
728,vide,vide,vide,mac reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
742,vide,vide,vide,mac reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23
752,vide,vide,vide,mac reconditionné,vide,vide,SEO vide,vide,vide,,25/07/23


# Export

In [ ]:
df.to_csv('liste_articles.csv', index = False)